In [1]:
import json
import pandas as pd
import requests
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
import time

In [17]:
# CSV 파일 읽기
df = pd.read_csv('tour_url.csv')
df

,s_tit,link1_href
0,성산일출봉(UNESCO 세계자연유산),https://www.visitjeju.net/kr/detail/view?conte...
1,우도(해양도립공원),https://www.visitjeju.net/kr/detail/view?conte...
2,사려니숲길,https://www.visitjeju.net/kr/detail/view?conte...
3,카멜리아힐,https://www.visitjeju.net/kr/detail/view?conte...
4,협재해수욕장,https://www.visitjeju.net/kr/detail/view?conte...
...,...,...
1483,황우럭만화천국사회적협동조합,https://www.visitjeju.net/kr/detail/view?conte...
1484,효명사,https://www.visitjeju.net/kr/detail/view?conte...
1485,후포해변,https://www.visitjeju.net/kr/detail/view?conte...
1486,휘닉스 르쏠레이테라피,https://www.visitjeju.net/kr/detail/view?conte...


In [18]:
# 칼럼명 변경
df = df.rename(columns={'s_tit': 'name', 'link1_href': 'url'})
df

,name,url
0,성산일출봉(UNESCO 세계자연유산),https://www.visitjeju.net/kr/detail/view?conte...
1,우도(해양도립공원),https://www.visitjeju.net/kr/detail/view?conte...
2,사려니숲길,https://www.visitjeju.net/kr/detail/view?conte...
3,카멜리아힐,https://www.visitjeju.net/kr/detail/view?conte...
4,협재해수욕장,https://www.visitjeju.net/kr/detail/view?conte...
...,...,...
1483,황우럭만화천국사회적협동조합,https://www.visitjeju.net/kr/detail/view?conte...
1484,효명사,https://www.visitjeju.net/kr/detail/view?conte...
1485,후포해변,https://www.visitjeju.net/kr/detail/view?conte...
1486,휘닉스 르쏠레이테라피,https://www.visitjeju.net/kr/detail/view?conte...


In [19]:
url = df['url']

In [20]:
url

0       https://www.visitjeju.net/kr/detail/view?conte...
1       https://www.visitjeju.net/kr/detail/view?conte...
2       https://www.visitjeju.net/kr/detail/view?conte...
3       https://www.visitjeju.net/kr/detail/view?conte...
4       https://www.visitjeju.net/kr/detail/view?conte...
                              ...                        
1483    https://www.visitjeju.net/kr/detail/view?conte...
1484    https://www.visitjeju.net/kr/detail/view?conte...
1485    https://www.visitjeju.net/kr/detail/view?conte...
1486    https://www.visitjeju.net/kr/detail/view?conte...
1487    https://www.visitjeju.net/kr/detail/view?conte...
Name: url, Length: 1488, dtype: object

In [21]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =url['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[0:300]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df1 = pd.DataFrame(data_list)

# 데이터 csv 저장
df1.to_csv('tour_info1.csv', index=False, encoding='utf-8')


100%|██████████| 300/300 [22:45<00:00,  4.55s/it]


In [22]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =url['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[300:600]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df2 = pd.DataFrame(data_list)

# 데이터 csv 저장
df2.to_csv('tour_info2.csv', index=False, encoding='utf-8')


100%|██████████| 300/300 [22:54<00:00,  4.58s/it]


In [23]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =url['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[600:900]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df3 = pd.DataFrame(data_list)

# 데이터 csv 저장
df3.to_csv('tour_info3.csv', index=False, encoding='utf-8')


100%|██████████| 300/300 [23:19<00:00,  4.67s/it]


In [27]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =url['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[900:1200]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df4 = pd.DataFrame(data_list)

# 데이터 csv 저장
df4.to_csv('tour_info4.csv', index=False, encoding='utf-8')


 69%|██████▊   | 206/300 [13:55<05:59,  3.83s/it]

In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =url['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[1200:]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df5 = pd.DataFrame(data_list)

# 데이터 csv 저장
df5.to_csv('tour_info5.csv', index=False, encoding='utf-8')


In [ ]:
merged_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [ ]:
merged_df.to_csv('tour_all_list.csv',index= False, encoding= 'utf-8')